In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM1 = pd.read_csv ('T1DM.csv')
dfTDM1 = dfTDM1.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
#dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Converti tutte le colonne float64 in int
dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM1, test_size=0.5, stratify=dfTDM1['DIABTYPE1'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("Modello creato")

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=4)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE1'].values
X_train = train_data.drop(columns=['DIABTYPE1'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE1']).copy()
    y_true = fold_val['DIABTYPE1'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                    y_pred.append(prediction['DIABTYPE1'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')

# Addestrare il modello finale sull'intero set di training
try:
    final_model = BayesianNetwork(best_model_structure.edges())
    final_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello finale: {e}')
    raise
print("Modello finale addestrato")

# Creare un oggetto per l'inferenza
final_inference = VariableElimination(final_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE1']).copy()
y_test_true = test_data['DIABTYPE1'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = final_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                y_pred.append(prediction['DIABTYPE1'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')


In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM1 = pd.read_csv ('T1DM.csv')
dfTDM1 = dfTDM1.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
#dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Converti tutte le colonne float64 in int
dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM1, test_size=0.1, stratify=dfTDM1['DIABTYPE1'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=5)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Variabili per salvare il miglior fold e il suo modello
best_fold = None
best_model = None
#best_accuracy = -1  # Inizialmente un valore basso, che verrà aggiornato
best_f1 = -1

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE1'].values
X_train = train_data.drop(columns=['DIABTYPE1'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        print(fold_model)
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE1']).copy()
    y_true = fold_val['DIABTYPE1'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                    y_pred.append(prediction['DIABTYPE1'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    
    # Salvare il modello con la migliore accuratezza
    #if accuracy > best_accuracy:
    #    best_accuracy = accuracy
    #    best_fold = fold
    #    best_model = fold_model  # Salva il miglior modello

    # Salvare il modello con il miglior F1 score
    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold
        best_model = fold_model  # Salva il miglior modello
    
# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')

# Stampare il miglior fold e l'accuratezza associata
#print(f'\nMiglior modello nel fold {best_fold} con Accuratezza = {best_accuracy:.4f}')
print(f'\nMiglior modello nel fold {best_fold} con F1 Score = {best_f1:.4f}')

# Utilizzare il miglior modello per fare inferenza sul set di test
final_inference = VariableElimination(best_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE1']).copy()
y_test_true = test_data['DIABTYPE1'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = best_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                y_pred.append(prediction['DIABTYPE1'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')


In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM2 = pd.read_csv('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
#dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Converti tutte le colonne float64 in int
dfTDM2 = dfTDM2.astype({col: 'int' for col in dfTDM2.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM2, test_size=0.5, stratify=dfTDM2['DIABTYPE2'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=4)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Variabili per salvare il miglior fold e il suo modello
best_fold = None
best_model = None
#best_accuracy = -1  # Inizialmente un valore basso, che verrà aggiornato
best_f1 = -1

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE2'].values
X_train = train_data.drop(columns=['DIABTYPE2'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        print(fold_model)
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE2']).copy()
    y_true = fold_val['DIABTYPE2'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                    y_pred.append(prediction['DIABTYPE2'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    
    # Salvare il modello con la migliore accuratezza
    #if accuracy > best_accuracy:
    #    best_accuracy = accuracy
    #    best_fold = fold
    #    best_model = fold_model  # Salva il miglior modello

    # Salvare il modello con il miglior F1 score
    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold
        best_model = fold_model  # Salva il miglior modello
    
# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')

# Stampare il miglior fold e l'accuratezza associata
#print(f'\nMiglior modello nel fold {best_fold} con Accuratezza = {best_accuracy:.4f}')
print(f'\nMiglior modello nel fold {best_fold} con F1 Score = {best_f1:.4f}')

# Utilizzare il miglior modello per fare inferenza sul set di test
final_inference = VariableElimination(best_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE2']).copy()
y_test_true = test_data['DIABTYPE2'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = best_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                y_pred.append(prediction['DIABTYPE2'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')


In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM1 = pd.read_csv ('T1DM.csv')
dfTDM1 = dfTDM1.drop('Unnamed: 0', axis = 1)

# Definire la struttura della rete bayesiana
dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})
hc = HillClimbSearch(dfTDM1)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(dfTDM1), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")

# Cross-validation
skf = StratifiedKFold(n_splits=5)  # Imposta il numero di fold
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target
y = dfTDM1['DIABTYPE1'].values
X = dfTDM1.drop(columns=['DIABTYPE1'])

# Itera attraverso le suddivisioni di StratifiedKFold
for train_index, test_index in skf.split(X, y):
    train_data = dfTDM1.iloc[train_index]
    df_test_data = dfTDM1.iloc[test_index]

    # Creare il modello con la struttura appresa
    try:
        model = BayesianNetwork(best_model_structure.edges())
    except Exception as e:
        print(f'Errore nella creazione del modello di rete bayesiana: {e}')
        raise
    print(model)
    print("Modello creato")

    # Addestrare la rete bayesiana con i dati di training
    try:
        model.fit(train_data, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(model)

    # Separare le features e la variabile target per il test
    X_test = df_test_data.drop(columns=['DIABTYPE1']).copy()
    y_true = df_test_data['DIABTYPE1'].values
    y_pred_total = []

    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_test), 100):
        end = start + 100
        df_blocco = X_test.iloc[start:end]
       
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
           
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
           
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE1'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE1' in prediction:
                    y_pred.append(prediction['DIABTYPE1'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
       
        y_pred_total.extend(y_pred)

    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)

    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro')
    recall = recall_score(y_true_final, y_pred_final, average='macro')
    f1 = f1_score(y_true_final, y_pred_final, average='macro')

    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche
print(f'Accuratezza media: {np.mean(accuracies)}')
print(f'Precisione media: {np.mean(precisions)}')
print(f'Recall medio: {np.mean(recalls)}')
print(f'F1 Score medio: {np.mean(f1s)}')

In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM2 = pd.read_csv ('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

# Definire la struttura della rete bayesiana
dfTDM2 = dfTDM2.astype({col: 'int' for col in dfTDM2.select_dtypes(include='float64').columns})
hc = HillClimbSearch(dfTDM2)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(dfTDM2), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if best_model_structure.edges() is None:
    raise ValueError("La struttura della rete appresa è vuota.")

# Cross-validation
skf = StratifiedKFold(n_splits=5)  # Imposta il numero di fold
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target
y = dfTDM2['DIABTYPE2'].values
X = dfTDM2.drop(columns=['DIABTYPE2'])

# Itera attraverso le suddivisioni di StratifiedKFold
for train_index, test_index in skf.split(X, y):
    train_data = dfTDM2.iloc[train_index]
    df_test_data = dfTDM2.iloc[test_index]

    # Creare il modello con la struttura appresa
    try:
        model = BayesianNetwork(best_model_structure.edges())
    except Exception as e:
        print(f'Errore nella creazione del modello di rete bayesiana: {e}')
        raise
    print(model)
    print("Modello creato")

    # Addestrare la rete bayesiana con i dati di training
    try:
        model.fit(train_data, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(model)

    # Separare le features e la variabile target per il test
    X_test = df_test_data.drop(columns=['DIABTYPE2']).copy()
    y_true = df_test_data['DIABTYPE2'].values
    y_pred_total = []

    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_test), 100):
        end = start + 100
        df_blocco = X_test.iloc[start:end]
       
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
           
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
           
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                    y_pred.append(prediction['DIABTYPE2'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
       
        y_pred_total.extend(y_pred)

    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)

    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro')
    recall = recall_score(y_true_final, y_pred_final, average='macro')
    f1 = f1_score(y_true_final, y_pred_final, average='macro')

    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche
print(f'Accuratezza media: {np.mean(accuracies)}')
print(f'Precisione media: {np.mean(precisions)}')
print(f'Recall medio: {np.mean(recalls)}')
print(f'F1 Score medio: {np.mean(f1s)}')

In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM1 = pd.read_csv ('T1DM.csv')
dfTDM1 = dfTDM1.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
dfTDM1 = dfTDM1.astype({col: 'int' for col in dfTDM1.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM1, test_size=0.2, stratify=dfTDM1['DIABTYPE1'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(dfTDM1)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("Modello creato")

In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM2 = pd.read_csv ('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
dfTDM2 = dfTDM2.astype({col: 'int' for col in dfTDM2.select_dtypes(include='float64').columns})
print('qui')
# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM2, test_size=0.5, stratify=dfTDM2['DIABTYPE2'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("Modello creato")

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=4)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE2'].values
X_train = train_data.drop(columns=['DIABTYPE2'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE2']).copy()
    y_true = fold_val['DIABTYPE2'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                    y_pred.append(prediction['DIABTYPE2'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')

# Addestrare il modello finale sull'intero set di training
try:
    final_model = BayesianNetwork(best_model_structure.edges())
    final_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello finale: {e}')
    raise
print("Modello finale addestrato")

# Creare un oggetto per l'inferenza
final_inference = VariableElimination(final_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE2']).copy()
y_test_true = test_data['DIABTYPE2'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = final_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                y_pred.append(prediction['DIABTYPE2'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')


In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM2 = pd.read_csv ('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
dfTDM2 = dfTDM2.astype({col: 'float32' for col in dfTDM2.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM2, test_size=0.2, stratify=dfTDM2['DIABTYPE2'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Creare il modello con la struttura appresa
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')
    raise
print(model)
print("Modello creato")

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE2'].values
X_train = train_data.drop(columns=['DIABTYPE2'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE2']).copy()
    y_true = fold_val['DIABTYPE2'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                    y_pred.append(prediction['DIABTYPE2'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')



In [ ]:
# Addestrare il modello finale sull'intero set di training
try:
    final_model = BayesianNetwork(best_model_structure.edges())
    final_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello finale: {e}')
    raise
print("Modello finale addestrato")

# Creare un oggetto per l'inferenza
final_inference = VariableElimination(final_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE2']).copy()
y_test_true = test_data['DIABTYPE2'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = final_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                y_pred.append(prediction['DIABTYPE2'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')


In [ ]:
from pgmpy.estimators import HillClimbSearch, K2Score, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

dfTDM2 = pd.read_csv ('T2DM.csv')
dfTDM2 = dfTDM2.drop('Unnamed: 0', axis = 1)

# Convertire i tipi di dati appropriati
dfTDM2 = dfTDM2.astype({col: 'float32' for col in dfTDM2.select_dtypes(include='float64').columns})

# Suddividere il dataset in 80% training e 20% test
train_data, test_data = train_test_split(dfTDM2, test_size=0.2, stratify=dfTDM2['DIABTYPE2'], random_state=42)

# Definire la struttura della rete bayesiana utilizzando i dati di training
hc = HillClimbSearch(train_data)
best_model_structure = hc.estimate(max_iter=100000, scoring_method=K2Score(train_data), max_indegree=2)
print("Archi della struttura appresa:", best_model_structure.edges())

# Verifica che best_model_structure.edges() non sia None
if not best_model_structure.edges():
    raise ValueError("La struttura della rete appresa è vuota.")

# Creare il modello con la struttura appres
try:
    model = BayesianNetwork(best_model_structure.edges())
except Exception as e:
    print(f'Errore nella creazione del modello di rete bayesiana: {e}')

# Cross-validation sul set di training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Imposta il numero di fold e lo shuffle
accuracies = []
precisions = []
recalls = []
f1s = []

# Separare le feature e la variabile target nel set di training
y_train = train_data['DIABTYPE2'].values
X_train = train_data.drop(columns=['DIABTYPE2'])

# Itera attraverso le suddivisioni di StratifiedKFold
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train), 1):
   raise
    fold_train = train_data.iloc[train_index]
    fold_val = train_data.iloc[val_index]
    
    # Definire e addestrare il modello per ogni fold
    try:
        fold_model = BayesianNetwork(best_model_structure.edges())
        fold_model.fit(fold_train, estimator=MaximumLikelihoodEstimator)
    except Exception as e:
        print(f'Errore durante l\'addestramento del modello nel fold {fold}: {e}')
        continue

    # Creare un oggetto per l'inferenza
    inference = VariableElimination(fold_model)
    
    # Separare le features e la variabile target per la validazione
    X_val = fold_val.drop(columns=['DIABTYPE2']).copy()
    y_true = fold_val['DIABTYPE2'].values
    y_pred_total = []
    
    # Iterare il dataframe a blocchi di 100 righe
    for start in range(0, len(X_val), 100):
        end = start + 100
        df_blocco = X_val.iloc[start:end]
        y_pred = []
        for index, row in df_blocco.iterrows():
            evidence = row.to_dict()
            # Validazione delle evidenze
            try:
                for node, value in evidence.items():
                    cpd = fold_model.get_cpds(node)
                    valid_states = cpd.state_names[node]
                    if value not in valid_states:
                        raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
            except Exception as e:
                print(f'Validazione evidenza fallita per index {index}: {e}')
                y_pred.append(None)
                continue
            # Inferenza
            try:
                prediction = inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
                if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                    y_pred.append(prediction['DIABTYPE2'])
                else:
                    y_pred.append(None)
            except Exception as e:
                print(f'Errore al processing dell\'indice {index}: {e}')
                y_pred.append(None)
        y_pred_total.extend(y_pred)
    
    # Filtrare le previsioni valide
    y_pred_final = []
    y_true_final = []
    for pred, true in zip(y_pred_total, y_true):
        if pred is not None:
            y_pred_final.append(pred)
            y_true_final.append(true)
    
    # Calcolare le metriche di valutazione
    accuracy = accuracy_score(y_true_final, y_pred_final)
    precision = precision_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    recall = recall_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    f1 = f1_score(y_true_final, y_pred_final, average='macro', zero_division=0)
    
    print(f'Fold {fold}: Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1 Score={f1}')
    
    # Aggiungere le metriche agli elenchi
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calcolare e stampare le medie delle metriche della cross-validation
print(f'\nAccuratezza media (CV): {np.mean(accuracies):.4f}')
print(f'Precisione media (CV): {np.mean(precisions):.4f}')
print(f'Recall medio (CV): {np.mean(recalls):.4f}')
print(f'F1 Score medio (CV): {np.mean(f1s):.4f}')




In [ ]:
# Addestrare il modello finale sull'intero set di training
try:
    final_model = BayesianNetwork(best_model_structure.edges())
    final_model.fit(train_data, estimator=MaximumLikelihoodEstimator)
except Exception as e:
    print(f'Errore durante l\'addestramento del modello finale: {e}')
    raise
print("Modello finale addestrato")

# Creare un oggetto per l'inferenza
final_inference = VariableElimination(final_model)

# Separare le features e la variabile target per il test
X_test = test_data.drop(columns=['DIABTYPE2']).copy()
y_test_true = test_data['DIABTYPE2'].values
y_test_pred_total = []

# Iterare il dataframe di test a blocchi di 100 righe
for start in range(0, len(X_test), 100):
    end = start + 100
    df_blocco = X_test.iloc[start:end]
    y_pred = []
    for index, row in df_blocco.iterrows():
        evidence = row.to_dict()
        # Validazione delle evidenze
        try:
            for node, value in evidence.items():
                cpd = final_model.get_cpds(node)
                valid_states = cpd.state_names[node]
                if value not in valid_states:
                    raise ValueError(f"Lo stato '{value}' non è valido per il nodo '{node}'")
        except Exception as e:
            print(f'Validazione evidenza fallita per index {index}: {e}')
            y_pred.append(None)
            continue
        # Inferenza
        try:
            prediction = final_inference.map_query(variables=['DIABTYPE2'], evidence=evidence)
            if isinstance(prediction, dict) and 'DIABTYPE2' in prediction:
                y_pred.append(prediction['DIABTYPE2'])
            else:
                y_pred.append(None)
        except Exception as e:
            print(f'Errore al processing dell\'indice {index}: {e}')
            y_pred.append(None)
    y_test_pred_total.extend(y_pred)

# Filtrare le previsioni valide per il test
y_test_pred_final = []
y_test_true_final = []
for pred, true in zip(y_test_pred_total, y_test_true):
    if pred is not None:
        y_test_pred_final.append(pred)
        y_test_true_final.append(true)

# Calcolare le metriche di valutazione sul set di test
test_accuracy = accuracy_score(y_test_true_final, y_test_pred_final)
test_precision = precision_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_recall = recall_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)
test_f1 = f1_score(y_test_true_final, y_test_pred_final, average='macro', zero_division=0)

# Stampare le metriche di valutazione sul set di test
print(f'\nAccuratezza sul set di test: {test_accuracy:.4f}')
print(f'Precisione sul set di test: {test_precision:.4f}')
print(f'Recall sul set di test: {test_recall:.4f}')
print(f'F1 Score sul set di test: {test_f1:.4f}')